In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", 40)
import numpy as np
import re
import spacy
import nltk
from nltk import FreqDist
import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

## Load  Data

In [3]:
data = pd.read_csv("cryptodata_test_lemmatized", sep=",", index_col=0)

In [5]:
data.shape

(500, 6)

In [6]:
data.head()

,body,stopword_reviews,body_as_str,tokens,review_lemmatized,body_lemmatized
time,,,,,,
1543622368,CoinSpeakerProBit: Professional Digi...,"['coinspeakerprobit', 'professional'...",CoinSpeakerProBit: Professional Digi...,"['coinspeakerprobit', 'professional'...","['coinspeakerprob', 'professional', ...","['coinspeakerprob', 'professional', ..."
1543620634,Today in crypto heard rumors of Sato...,"['today', 'crypto', 'heard', 'rumors...",Today in crypto heard rumors of Sato...,"['today', 'crypto', 'heard', 'rumors...","['today', 'crypto', 'heard', 'rumor'...","['today', 'crypto', 'heard', 'rumor'..."
1543620300,Crashes in the crypto market resulte...,"['crashes', 'crypto', 'market', 'res...",Crashes in the crypto market resulte...,"['crashes', 'the', 'crypto', 'market...","['crash', 'the', 'crypto', 'market',...","['crash', 'the', 'crypto', 'market',..."
1543620202,SEC Chairman Jay Clayton has claimed...,"['sec', 'chairman', 'jay', 'clayton'...",SEC Chairman Jay Clayton has claimed...,"['sec', 'chairman', 'jay', 'clayton'...","['sec', 'chairman', 'jay', 'clayton'...","['sec', 'chairman', 'jay', 'clayton'..."
1543618813,There are four different Ethereum wo...,"['four', 'different', 'ethereum', 'w...",There are four different Ethereum wo...,"['there', 'are', 'four', 'different'...","['ther', 'are', 'four', 'different',...","['ther', 'are', 'four', 'different',..."


In [7]:
#the lemmantized_reviews are interpreted as a string by the read_csv command. We convert it into a list. 
data['stopword_reviews'] = data['stopword_reviews'].str.replace(r"[\[\]\']", "").str.split(", ")

In [8]:
data['stopword_reviews'] #checking if it's a list

time
1543622368    [coinspeakerprobit, professional, di...
1543620634    [today, crypto, heard, rumors, satos...
1543620300    [crashes, crypto, market, resulted, ...
1543620202    [sec, chairman, jay, clayton, claime...
1543618813    [four, different, ethereum, working,...
1543617903    [nov, 29, sirin, labs, announced, co...
1543616700    [switzerlands, oldest, university, u...
1543616305    [newer, hbus, cryptocurrency, exchan...
1543616240    [met, u, securities, exchange, commi...
1543615852    [latest, ethereum, 1x, meeting, cond...
1543615242    [chinese, cryptocurrency, mining, ha...
1543615229    [blockchain, phones, therage, right,...
1543614900    [check, point, research, cybersecuri...
1543613835    [belarus, following, model, set, for...
1543612078    [casual, gamers, unite, stay, thats,...
1543611643    [new, report, coinshares, bitcoin, m...
1543611641    [one, co, founders, second, largest,...
1543611625    [co, founder, ethereum, vitalik, but...
1543611600    [2017, ea

In [9]:
data.dropna(subset=['stopword_reviews'], inplace=True)

In [10]:
data.shape

(500, 6)

## Remove words

In [11]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/elliot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
import stopwords 

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(lang)? (__init__.py, line 84)

In [13]:
# We list the words we want to get rid of"
# list_stop_words = ["i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now"]
list_stop_words = ["&", ";", "2", "get","de", "got", "went","much","well","day","stay","even","could","area","also","uk","us","one","center","really","would","4","great","good","cottage","parc","parcs","park","parks"]

In [14]:
def remove_stop_word(x, list_stop_words=list_stop_words):
    clean_x = [l for l in x if l not in list_stop_words]
    return clean_x

In [15]:
data['stopword_reviews'] = data['stopword_reviews'].apply(remove_stop_word)

In [16]:
data.head()

,body,stopword_reviews,body_as_str,tokens,review_lemmatized,body_lemmatized
time,,,,,,
1543622368,CoinSpeakerProBit: Professional Digi...,"[coinspeakerprobit, professional, di...",CoinSpeakerProBit: Professional Digi...,"['coinspeakerprobit', 'professional'...","['coinspeakerprob', 'professional', ...","['coinspeakerprob', 'professional', ..."
1543620634,Today in crypto heard rumors of Sato...,"[today, crypto, heard, rumors, satos...",Today in crypto heard rumors of Sato...,"['today', 'crypto', 'heard', 'rumors...","['today', 'crypto', 'heard', 'rumor'...","['today', 'crypto', 'heard', 'rumor'..."
1543620300,Crashes in the crypto market resulte...,"[crashes, crypto, market, resulted, ...",Crashes in the crypto market resulte...,"['crashes', 'the', 'crypto', 'market...","['crash', 'the', 'crypto', 'market',...","['crash', 'the', 'crypto', 'market',..."
1543620202,SEC Chairman Jay Clayton has claimed...,"[sec, chairman, jay, clayton, claime...",SEC Chairman Jay Clayton has claimed...,"['sec', 'chairman', 'jay', 'clayton'...","['sec', 'chairman', 'jay', 'clayton'...","['sec', 'chairman', 'jay', 'clayton'..."
1543618813,There are four different Ethereum wo...,"[four, different, ethereum, working,...",There are four different Ethereum wo...,"['there', 'are', 'four', 'different'...","['ther', 'are', 'four', 'different',...","['ther', 'are', 'four', 'different',..."


In [17]:
data.shape

(500, 6)

## Building an LDA model

In [18]:
dictionary = corpora.Dictionary(data['stopword_reviews'])

In [19]:
len(dictionary)

4669

In [20]:
doc_term_matrix = [dictionary.doc2bow(rev) for rev in data['stopword_reviews']]

In [23]:
import warnings
warnings.filterwarnings('ignore')

# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
num_topics = 4
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, 
                num_topics=num_topics, 
                alpha=[0.0001] * num_topics, 
                eta=[0.0001] * len(dictionary),
                chunksize=2000,
                passes=4,
                random_state=100,
               )

In [24]:
lda_model.print_topics(num_words=5)

[(0,
  '0.020*"first" + 0.017*"post" + 0.016*"appeared" + 0.013*"bitcoin" + 0.012*"cryptocurrency"'),
 (1,
  '0.032*"bitcoin" + 0.020*"" + 0.014*"first" + 0.013*"crypto" + 0.013*"market"'),
 (2,
  '0.017*"first" + 0.015*"blockchain" + 0.015*"" + 0.014*"bitcoin" + 0.013*"appeared"'),
 (3,
  '0.038*"$" + 0.018*"price" + 0.017*"bitcoin" + 0.015*"first" + 0.014*"crypto"')]

## Topic visualization

In [25]:
# Visualize the topics
pd.options.display.max_colwidth = 2000
viz = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary, mds='tsne')

In [26]:
pyLDAvis.enable_notebook()
viz
#pyLDAvis.show(viz)

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1     -324.249390 -289.197083       1        1  38.316200
0     -200.549118 -570.859375       2        1  26.021770
2      -42.603127 -165.460297       3        1  19.008968
3       81.096878 -447.122253       4        1  16.653061, topic_info=     Category        Freq            Term       Total  loglift  logprob
361   Default  272.000000               $  272.000000  30.0000  30.0000
260   Default  159.000000           price  159.000000  29.0000  29.0000
913   Default   42.000000          amazon   42.000000  28.0000  28.0000
109   Default  175.000000      blockchain  175.000000  27.0000  27.0000
72    Default   82.000000             ccn   82.000000  26.0000  26.0000
1259  Default   48.000000         support   48.000000  25.0000  25.0000
97    Default   68.000000             sec   68.000000  24.0000  24.0000
1465  Default   36.000000      resistance   36.000000  23.0000  23.0000
934   Default   47.000000             usd   47.000000  22.0000  22.0000
955   Default   21.000000          khaled   21.000000  21.0000  21.0000
1123  Default   14.000000        nakamoto   14.000000  20.0000  20.0000
122   Default   75.000000        platform   75.000000  19.0000  19.0000
947   Default   42.000000               0   42.000000  18.0000  18.0000
40    Default  333.000000           first  333.000000  17.0000  17.0000
603   Default   46.000000         million   46.000000  16.0000  16.0000
54    Default   14.000000         satoshi   14.000000  15.0000  15.0000
46    Default   62.000000          mining   62.000000  14.0000  14.0000
541   Default   19.000000              dj   19.000000  13.0000  13.0000
549   Default   19.000000      mayweather   19.000000  12.0000  12.0000
543   Default   19.000000           floyd   19.000000  11.0000  11.0000
587   Default   12.000000             120   12.000000  10.0000  10.0000
178   Default   15.000000      university   15.000000   9.0000   9.0000
1034  Default   19.000000           trend   19.000000   8.0000   8.0000
37    Default  226.000000          crypto  226.000000   7.0000   7.0000
607   Default   12.000000            rice   12.000000   6.0000   6.0000
1227  Default   33.000000         bullish   33.000000   5.0000   5.0000
1032  Default   21.000000            term   21.000000   4.0000   4.0000
1355  Default   24.000000             zec   24.000000   3.0000   3.0000
553   Default   19.000000            paid   19.000000   2.0000   2.0000
589   Default   11.000000       arisebank   11.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
956    Topic4    8.118007             pay   11.247478   1.4665  -6.0439
260    Topic4   61.136250           price  159.706879   0.8323  -4.0249
2610   Topic4    6.001902           steem    8.111206   1.4914  -6.3459
603    Topic4   20.984663         million   46.575996   0.9953  -5.0942
1465   Topic4   17.432028      resistance   36.479904   1.0541  -5.2797
934    Topic4   19.368164             usd   47.400665   0.8976  -5.1744
1227   Topic4   15.296698         bullish   33.531033   1.0077  -5.4104
1549   Topic4   10.798373               3   20.313902   1.1607  -5.7586
609    Topic4    8.944323         startup   15.240002   1.2597  -5.9470
37     Topic4   48.247757          crypto  226.865753   0.2446  -4.2616
592    Topic4   15.553089             ceo   38.301048   0.8914  -5.3937
1002   Topic4   14.473680        analysis   34.275711   0.9305  -5.4657
947    Topic4   15.790325               0   42.428230   0.8042  -5.3786
35     Topic4   58.203423         bitcoin  436.091064  -0.2213  -4.0740
40     Topic4   49.702103           first  333.106873  -0.1098  -4.2319
50     Topic4   43.408508            post  279.346405  -0.0692  -4.3673
34     Topic4   40.496021        appeared  267.393829  -0.0949  -4.4368
301    Topic4   14.755003         newsbtc   47.098320   0.6319  -5.4464
1259   To

In [28]:
data.to_csv(r'crpytodata_test_post_lda.csv', index = False)